In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import requests

In [2]:
def get_matches_data(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    string = soup.find_all('script')[4].string
    start_idx = string.index("('")
    stop_idx = string.index("')")
    string = string[start_idx+2:stop_idx]
    data = string.encode('utf8').decode('unicode_escape')
    json_data = json.loads(data)
    return pd.DataFrame(json_data)
    

In [3]:
data = get_matches_data("https://understat.com/player/2370")

In [4]:
data.head(5)

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,id,season,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup
0,1,5,0.7578337788581848,76,FW,Real Madrid,Levante,6,0,2022-05-12,17490,2021,535045,1.4455233812332153,1,4,1,0.7578337788581848,2.1534039974212646,0.4140799939632416
1,1,1,0.5057440400123596,26,Sub,Real Madrid,Espanyol,4,0,2022-04-30,17471,2021,530593,0,0,0,1,0.5057440400123596,0.786812961101532,0.28106892108917236
2,0,5,1.6936463117599487,90,FW,Osasuna,Real Madrid,1,3,2022-04-20,17463,2021,528146,0.678108274936676,0,4,0,0.20709101855754852,0.9920371770858765,0.8843826651573181
3,1,7,0.9199050664901733,90,FW,Sevilla,Real Madrid,2,3,2022-04-17,17453,2021,527392,0,0,0,1,0.9199050664901733,0.9199050664901733,0.5905827879905701
4,0,4,0.25150060653686523,75,FW,Real Madrid,Getafe,2,0,2022-04-09,17441,2021,524912,0.10142123699188232,0,3,0,0.25150060653686523,0.2579692006111145,0.037565723061561584


In [5]:
def transform_data(df):
    new_df = df.copy()
    for col in new_df.columns:
        if col in ['position', 'h_team', 'a_team']:
            new_df[col] = new_df[col].astype(str)
        elif col in ['goals', 'shots', 'time', 'h_goals', 'a_goals', 'assists', 'key_passes', 'npg', 'season']:
            new_df[col] = new_df[col].astype(int)
        elif col in ['xG', 'xA', 'npxG', 'xGChain', 'xGBuildup']:
            new_df[col] = new_df[col].astype(float).round(2)
        elif col in 'date':
            new_df[col] = pd.to_datetime(new_df[col])
    
    new_df.drop(['id', 'roster_id', 'position'], axis=1, inplace=True)
    new_df.columns = ['Goals', 'Shots', 'XG', 'Minutes Played', 'Home Team', 'Away Team', 'Home Goals','Away Goals', 'Date', 'Season', 'XA','Assists', 'Key Passes', 'NPG', 'NPXG', 'XGChain', 'XGBuildup']
    
    return new_df
    
    

In [6]:
df = transform_data(data)

In [7]:
df

,Goals,Shots,XG,Minutes Played,Home Team,Away Team,Home Goals,Away Goals,Date,Season,XA,Assists,Key Passes,NPG,NPXG,XGChain,XGBuildup
0,1,5,0.76,76,Real Madrid,Levante,6,0,2022-05-12,2021,1.45,1,4,1,0.76,2.15,0.41
1,1,1,0.51,26,Real Madrid,Espanyol,4,0,2022-04-30,2021,0.00,0,0,1,0.51,0.79,0.28
2,0,5,1.69,90,Osasuna,Real Madrid,1,3,2022-04-20,2021,0.68,0,4,0,0.21,0.99,0.88
3,1,7,0.92,90,Sevilla,Real Madrid,2,3,2022-04-17,2021,0.00,0,0,1,0.92,0.92,0.59
4,0,4,0.25,75,Real Madrid,Getafe,2,0,2022-04-09,2021,0.10,0,3,0,0.25,0.26,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0,1,0.15,84,Villarreal,Real Madrid,0,2,2014-09-27,2014,0.53,1,1,0,0.15,0.75,0.08
251,0,2,0.11,59,Deportivo La Coruna,Real Madrid,2,8,2014-09-20,2014,0.91,2,3,0,0.11,1.04,0.07
252,0,3,0.14,63,Real Madrid,Atletico Madrid,1,2,2014-09-13,2014,0.00,0,0,0,0.14,0.14,0.00
253,0,2,0.43,90,Real Sociedad,Real Madrid,4,2,2014-08-31,2014,0.12,0,1,0,0.43,0.59,0.04


In [8]:
df.to_csv(".//Datasets//kb9_match_states_2014_2021.csv", index=False)